In [1]:
!pip install selenium
!pip install webdriver-manager

!pip install requests
!pip install pandas

In [2]:
#  Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.
from tqdm import tqdm, trange
import re
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [6]:
def tuplas_actores(id_pelicula):
    #accedemos a la página IMDB:
    #accedemos a la página IMDB:
    driver = webdriver.Chrome() #abrir el navegador
    driver.get("https://www.imdb.com/") # abrir pagina web
    driver.maximize_window() #maximizar ventana navegador
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click() # acepto las coockies
    sleep(3)
    #Buscamos pelicula por el id, en la barra buscador:

    busqueda = driver.find_element("css selector","#suggestion-search").send_keys(id_pelicula , Keys.ENTER)
    sleep(3)

    #quiero que haga scroll hasta donde ponga 'rparto principal'

    xpath_i='//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[1]'
    pyhton= driver.find_element(By.XPATH, xpath_i)
    driver.execute_script('arguments[0].scrollIntoView(true)', pyhton)

    #driver.execute_script("window.scrollTo(0,1500)")
    sleep(5)

        #ACCEDEMOS A 'REPARTO PRINCIPAL >'
    driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-title.ipc-title--base.ipc-title--section-title.ipc-title--on-textPrimary > div > a > h3 > svg').click()
    sleep(5)
    #except:
        #driver.execute_script("window.scrollTo(0,1000)") #si hay problemas con el scroll, probar con uno mas corto.
        #sleep(3)
        #ACCEDEMOS A 'REPARTO PRINCIPAL >'
        #driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[1]/div/a/h3').click()
        #sleep(5)
    

    lista_actores_peli_i=[]

    #ciclo for para los 10 primeros actores: range(1,11):
    for i in range(2,4):
        
        sleep(5)
        #accedemos a la informacion del actor/actriz:
        driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
        sleep(5)
        #seleccionamos desplegable, para acceder a biografia y premios:
        driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografia
        sleep(5)
        #acceso a la biografia
        driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
        sleep(5)
        
        # obtenemos el NOMBRE del actor
        nombre= driver.find_element("xpath",'//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[2]/hgroup/h2').text
        
        try:
            #obtenemos AÑO
            año_sin_limp=driver.find_element("xpath",'//*[@id="born"]/div/div/div/a[1]').text
            año=(re.findall('\d{4}', año_sin_limp))[0] #sacamos del texto cadena de texto con 4 digitos seguidos, con regex.
        except:
            año='NULL'
        sleep(3)

        driver.back()
        #volvemos a pagina anterior, de nuevo vamos al desplegable
        #acceso a los PREMIOS
        driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografiasleep(5)
        sleep(3)
        driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[4]/a').click()#acceso a la biografia
        sleep(5)
        #creamos ciclo for para sacar las los premios:

        try:
            premios=[]
            for i in range(1,6): #solo queremos 5 pelis
                premio_sin_limpiar=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[{i}]').text
                premio_limpio= re.sub('\n',' ', premio_sin_limpiar)
                premios.append(premio_limpio)
            sleep(6)
        except:
            premios='NULL'

        sleep(6)
        driver.back()
        sleep(3)

        #hacemos scroll hasta CONOCIDO_POR:
        xpath_d='//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[1]'
        pyhton_d= driver.find_element(By.XPATH, xpath_d)
        driver.execute_script('arguments[0].scrollIntoView(true)', pyhton_d)
        sleep(10)
        #extraemos no mas de 4 pelis por las que sea conocido:

        conocido_por=[]
        for i in range(1,5):
            try:
                peli_conocido=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[2]/div/div[2]/div[{i}]/div[2]/div[1]/a').text
                conocido_por.append(peli_conocido)
                sleep(6)
            except:
                continue
        sleep(10)
     
        #volvemos arriba:
        driver.execute_script("window.scrollTo(0,0)")
        sleep(10)
        #extraemos informacion que hace.

        try:
            que_hace=[]
            for z in range(1,4):
                try:
                    peli_hace=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li[{z}]').text
                    que_hace.append(peli_hace)
                    sleep(2)
                except:
                    continue
        except:
            que_hace='NULL'
        
        #sacamos tupla de cada actor
        lista_infor_actor=[nombre, año, que_hace, conocido_por,premios]
        tupla_x=tuple(lista_infor_actor)
        lista_actores_peli_i.append(tupla_x)#apendeamos a la lista de actores pelicula

        sleep(6)
        driver.back()
        sleep(3)
        #volvemos a la página principal 

    return lista_actores_peli_i #obtenemos una lista con tuplas informativas de los 10 principales actores.


In [7]:
prueba_2=tuplas_actores('tt0398286')
prueba_2

[('Mandy Moore',
  '1984',
  ['Reparto', 'Guion', 'Producción'],
  [],
  ['Critics Choice Awards 2023 Nominado Critics Choice Award Best Actress in a Drama Series This Is Us 2011 Nominado Critics Choice Award Best Song Enredados For the song "I See the Light". Compartido con: Zachary Levi · Alan Menken · Glenn Slater',
   'Primetime Emmy Awards 2019 Nominado Primetime Emmy Outstanding Lead Actress in a Drama Series This Is Us For playing: "Rebecca Pearson".',
   'Golden Globes, USA 2017 Nominado Golden Globe Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television This Is Us',
   "Kids' Choice Awards, USA 2000 Ganador Kids' Choice Award Favorite Rising Star Mandy Moore: I Wanna Be with You",
   'MTV Movie + TV Awards 2017 Nominado MTV Movie + TV Award Best Actor in a Show This Is Us 2002 Ganador MTV Movie Award Breakthrough Female Performance Un paseo para recordar']),
 ('Zachary Levi',
  '1980',
  ['Reparto', 'Producción', '

In [ ]:
PROBLEMA EN EL REGEX DEL TEXTO SI APARECE EEUU

In [ ]:
   #obtenemos el texto para filtrar informacion 'CONOCIDO POR' Y 'A QUE SE DEDICA': todos presentan la misma estructura por lo que filtramos por regex:
        driver.execute_script("window.scrollTo(0,1000)") #hacemos scroll para acceder al texto que necesitamos:
        sleep(6)
        #Obtenemos el texto:
        conocido_por = driver.find_element("xpath",'//*[@id="mini_bio_0"]/div/ul/div/div').text 
        try:   
        #limpiamos el texto:
            patron_a='\.'
            lista_strings_texto=re.split(patron_a,conocido_por) #dividimos el texto por '.', para trabajarlo por listas. Se obtiene una lista con dos item:

            que_hace=re.split(', conocido por |, conocida por ', lista_strings_texto[1]) # seleccionamos el texto que empiece por 'conocido por'
            que_hace_limpio=re.sub(' Es un | Es una', '', que_hace[0]) # seleccionamos el texto que empiece por un/una ya que este codigo se aplica a varios actores y actrices
        except:
            que_hace_limpio='NULL'  #Si ocurre una excepción, es porque no hay texto o informacion a extraer.  
        try:
            conocido_por_limpio=que_hace[1]#quitamos los espacios al principio y final.
        
        except:
            # Si ocurre una excepción 
            conocido_por_limpio='NULL' # Si ocurre una excepción, es porque no hay texto o informacion a extraer.

In [57]:
prueba_2[0]


('Mandy Moore',
 '1984',
 'UU',
 'NULL',
 ['Critics Choice Awards 2023 Nominado Critics Choice Award Best Actress in a Drama Series This Is Us 2011 Nominado Critics Choice Award Best Song Enredados For the song "I See the Light". Compartido con: Zachary Levi · Alan Menken · Glenn Slater',
  'Primetime Emmy Awards 2019 Nominado Primetime Emmy Outstanding Lead Actress in a Drama Series This Is Us For playing: "Rebecca Pearson".',
  'Golden Globes, USA 2017 Nominado Golden Globe Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television This Is Us',
  "Kids' Choice Awards, USA 2000 Ganador Kids' Choice Award Favorite Rising Star Mandy Moore: I Wanna Be with You",
  'MTV Movie + TV Awards 2017 Nominado MTV Movie + TV Award Best Actor in a Show This Is Us 2002 Ganador MTV Movie Award Breakthrough Female Performance Un paseo para recordar'])

In [6]:
lista_id_prueba=['tt0331312','tt0337882','tt0393049','tt0398286']
lista_prueba=[]
for id in lista_id_prueba:
    prueba=tuplas_actores(id)
    print(prueba)
    print('================================================')
print(lista_prueba)


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (294, -390)
  (Session info: chrome=124.0.6367.119)
Stacktrace:
	GetHandleVerifier [0x00007FF73AF91502+60802]
	(No symbol) [0x00007FF73AF0AC02]
	(No symbol) [0x00007FF73ADC7CE4]
	(No symbol) [0x00007FF73AE1E877]
	(No symbol) [0x00007FF73AE1C344]
	(No symbol) [0x00007FF73AE1980A]
	(No symbol) [0x00007FF73AE189E2]
	(No symbol) [0x00007FF73AE0AE8B]
	(No symbol) [0x00007FF73AE3AB7A]
	(No symbol) [0x00007FF73AE0A7C6]
	(No symbol) [0x00007FF73AE3AD90]
	(No symbol) [0x00007FF73AE5A224]
	(No symbol) [0x00007FF73AE3A923]
	(No symbol) [0x00007FF73AE08FEC]
	(No symbol) [0x00007FF73AE09C21]
	GetHandleVerifier [0x00007FF73B29411D+3217821]
	GetHandleVerifier [0x00007FF73B2D60B7+3488055]
	GetHandleVerifier [0x00007FF73B2CF03F+3459263]
	GetHandleVerifier [0x00007FF73B04B846+823494]
	(No symbol) [0x00007FF73AF15F9F]
	(No symbol) [0x00007FF73AF10EC4]
	(No symbol) [0x00007FF73AF11052]
	(No symbol) [0x0